# Running Parallel OSDT

In [2]:
# third-party imports
from time import time

# local imports
from lib.parallel_osdt_classifier import ParallelOSDTClassifier
from lib.data_processing import read_dataset

# Using COMPAS as an example
dataset = read_dataset('data/preprocessed/compas-binary.csv') 
(n, m) = dataset.shape
X = dataset.values[:n,:m-1]
y = dataset.values[:n,-1]

hyperparameters = {
    'regularization': 0.005, # Regularization coefficient which effects the penalty on model complexity

    'max_depth': float('Inf'), # User-specified limit on the model
    'max_time': 60, # User-specified limit on the runtime 

    'workers': 1, # Parameter that varies based on how much computational resource is available

    'visualize': True, # Toggle whether a rule-list visualization is rendered
    'verbose': False, # Toggle whether event messages are printed
    'log': False, # Toggle whether client processes log to logs/work_<id>.log files
    'profile': False, # Toggle Snapshots for Profiling Memory Usage
    
    'configuration': { # More configurations around toggling optimizations and prioritization options
        'priority_metric': 'uniform', # Decides how tasks are prioritized
        'deprioritization': 0.01, # Decides how much to push back a task if it has pending dependencies

        # Note that Leaf Permutation Bound (Theorem 6) is 
        # Toggles the assumption about objective independence when composing subtrees (Theorem 1)
        # Disabling this actually breaks convergence due to information loss
        'hierarchical_lowerbound': True, 
        # Toggles whether problems are pruned based on insufficient accuracy (compared to other results) (Lemma 2)
        'look_ahead': True,
        # Toggles whether a split is avoided based on insufficient support (proxy for accuracy gain) (Theorem 3)
        'support_lowerbound': True,
        # Toggles whether a split is avoided based on insufficient potential accuracy gain (Theorem 4)
        'incremental_accuracy_lowerbound': True,
        # Toggles whether a problem is pruned based on insufficient accuracy (in general) (Theorem 5)
        'accuracy_lowerbound': True,
        # Toggles whether problem equivalence is based solely on the capture set (Similar to Corollary 6)
        'capture_equivalence': True,
        # Hamming distance used to propagate bounding information of similar problems (Theorem 7 + some more...)
        "similarity_threshold": 0,
        # Toggles whether equivalent points contribute to the lowerbound (Proposition 8 and Theorem 9)
        'equivalent_point_lowerbound': True,

        # Toggles compression of dataset based on equivalent point aggregation
        'equivalent_point_compression': True,
        # Toggles whether asynchronous tasks can be cancelled after being issued
        'task_cancellation': True,
        # Toggles whether look_ahead prunes using objective upperbounds (This builds on top of look_ahead)
        'interval_look_ahead': True,
        # Cooldown timer (seconds) on synchornization operations
        'synchronization_cooldown': 0.01,
        # Cache Limit
        'cache_limit': float('Inf')
    }
}

start = time()
model = ParallelOSDTClassifier(**hyperparameters)
model.fit(X, y)
prediction = model.predict(X)
prediction = prediction.reshape(1, n)
print('Runtime: {} Seconds'.format(time() - start))
print('Prediction: \n{}'.format(prediction))
print('Training Accuracy: {}'.format(model.score(X, y)))
print('Visualization: \n{}'.format(model.model.visualization))

Starting Parallel OSDT
Problem Definition:
  Number of Samples: 6907
  Number of Unique Samples: 123
  Number of Features: 12
  Regularization Coefficient: 0.005
Execetion Resources:
  Number of Woker Processes: 1
  Number of Server Processes: 1
Algorithm Configurations:
  priority_metric = uniform
  deprioritization = 0.01
  hierarchical_lowerbound = True
  look_ahead = True
  support_lowerbound = True
  incremental_accuracy_lowerbound = True
  accuracy_lowerbound = True
  capture_equivalence = True
  similarity_threshold = 0
  equivalent_point_lowerbound = True
  equivalent_point_compression = True
  task_cancellation = True
  interval_look_ahead = True
  synchronization_cooldown = 0.01
  cache_limit = inf
Worker 0 Starting
Case: Recursive, Problem: ():111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111 => Result(optimizer=None, optimum=(0.3275713044737223, 0.46771898074417256))
Case: Recursive, Problem: (11, 'L'

Case: Recursive, Problem: (1, 'L', 10, 'L'):000011000100001011111010000110110100100011100111110101011000000111110111101011100110110111001101001111011111011110110101111 => Result(optimizer=None, optimum=(0.2233292312147097, 0.32554437527146374))
Case: Recursive, Problem: (1, 'L', 10, 'R'):000000100000000000000101000000000011000000010000001010000100000000001000010100010001001000100010100000100000100001001010000 => Result(optimizer=None, optimum=(0.07130954104531635, 0.0872354133487766))
Case: Downward, Problem: (1, 'L'):000011100100001011111111000110110111100011110111111111011100000111111111111111110111111111101111101111111111111111111111111 => Result(optimizer=None, optimum=(0.29463877226002605, 0.3151976255972202))
Case: Cached, Problem: (8, 'R'):000100000101000100010000000111000000000011001010000000010000000100000000000001000100000000001000000100000000001000000000000 => Result(optimizer=(None, 0), optimum=0.09317142029824815)
Case: Base, Problem: (10, 'R', 7, 'L'):00000000100000000000

Case: Base, Problem: (2, 'R', 11, 'R'):100010000010001001000000100000101000000000000101000000100010000001000100000000000000000000010000000000010000000000000000000 => Result(optimizer=(None, 1), optimum=0.008619516432604604)
Case: Base, Problem: (2, 'R', 8, 'R'):000100000101000100010000000101000000000010001000000000000000000000000000000000000000000000000000000000000000000000000000000 => Result(optimizer=(None, 0), optimum=0.04220862892717533)
Case: Base, Problem: (2, 'R', 7, 'L'):000000011001001010010000111001101110011110001101011000100011111001000100111100001001000000010000010000010000000000000000000 => Result(optimizer=(None, 1), optimum=0.019767627045026784)
Case: Base, Problem: (2, 'R', 6, 'L'):000000000000000000000000000000001000011000001100011000100001100001000000110000001001000000010000010000010000000000000000000 => Result(optimizer=(None, 1), optimum=0.010067323005646445)
Case: Recursive, Problem: (2, 'R', 6, 'R'):11111111111111111101110011110110011000011000000100000000001001100

Case: Cached, Problem: (3, 'L', 7, 'L'):000000000000000000000000000000000000000000000000000000000000000010101000000001000010101000001000001011101100010111011101111 => Result(optimizer=(None, 1), optimum=0.022084117561893733)
Case: Cached, Problem: (3, 'L', 8, 'R'):000000000000000000000000000000000000000000000000000000010000000000000000000001000100000000001000000100000000001000000000000 => Result(optimizer=(None, 0), optimum=0.047420732590125955)
Case: Cached, Problem: (3, 'L', 10, 'R'):000000000000000000000000000000000000000000000000000010000000000000001000000000010000001000000000100000100000100001001000000 => Result(optimizer=(None, 0), optimum=0.05379108151151006)
Case: Cached, Problem: (3, 'L', 9, 'R'):000000000000000000000000000000000000000000000000000100000000000000100000000000100000100000000001000001001001000010010000000 => Result(optimizer=(None, 0), optimum=0.037575647893441436)
Case: Cached, Problem: (3, 'L', 11, 'L'):00000000000000000000000000000000000000000000000000011001000

Case: Recursive, Problem: (2, 'L', 10, 'L', 8, 'L'):000000000000000000100010000000010000100000100000100101001000000010110011000010100010110111000101001011001111010110110101111 => Result(optimizer=None, optimum=(0.1493463153322716, 0.20103300998986537))
Case: Base, Problem: (2, 'L', 10, 'L', 7, 'L'):000000000000000000000000000000010000000001000000000001001000000110110010000011000010110101001100001011001100010110110101111 => Result(optimizer=(None, 1), optimum=0.024979730707977415)
Case: Recursive, Problem: (2, 'L', 10, 'L', 7, 'R'):000000000000000000100010000010000000100000100010100100010000000000000001000000100100000010000001000100000011001000000000000 => Result(optimizer=None, optimum=(0.18322498914145072, 0.2407029100912118))
Case: Base, Problem: (2, 'L', 10, 'L', 6, 'L'):000000000000000000000000000000000000000000000000000000000000000000010000000000000000010001000000001001000000010010100001010 => Result(optimizer=(None, 1), optimum=0.014410742724771972)
Case: Recursive, Problem: (2, 

...

Case: Pruned, Problem: (10, 'L', 5, 'R', 3, 'R'):110111010111101111111010110111111100110011101110010001101001000101010000100000000000010001000000000000000000000000000000000
Worker 0 Idle Result(optimizer=None, optimum=(0.35263862747936875, 0.35596858259736497))
Case: Pruned, Problem: (10, 'L', 5, 'R', 4, 'R'):110111010111101111111010110111111100110011101110110101111001000111110001100001100110110001001000001001000000010010000000000
Worker 0 Idle Result(optimizer=None, optimum=(0.35263862747936875, 0.35596858259736497))
Case: Pruned, Problem: (10, 'L', 5, 'R', 4, 'R', 0, 'L'):000000000010100101001000110101100100100010100010000001101001000101000001100000100110110001001000000000000000010010000000000
Worker 0 Idle Result(optimizer=None, optimum=(0.35263862747936875, 0.35596858259736497))
Case: Pruned, Problem: (10, 'L', 5, 'R', 4, 'R', 1, 'L', 6, 'L'):000000000000000000000000000000000000000000000100010000000000000001010000100000000000010001000000001001000000010010000000000
Worker 0 Idle Res

Case: Pruned, Problem: (8, 'L', 5, 'R', 2, 'L'):000000000000000000100011000000010001100000110000100111001100000010111001000000110010111011100001101001100111110011000101000
Worker 0 Idle Result(optimizer=None, optimum=(0.35263862747936875, 0.35596858259736497))
Case: Pruned, Problem: (8, 'L', 6, 'R', 0, 'L'):000000000010110001001100111000100110100000110000000001001110011000000101001100110010101100000110000000000011100100011100001
Worker 0 Idle Result(optimizer=None, optimum=(0.35263862747936875, 0.35596858259736497))
Case: Pruned, Problem: (8, 'L', 5, 'R', 0, 'L'):000000000010110001001100111000100110100000110000000001101101100001000001110000110010111001100000000000000011110011000100000
Worker 0 Idle Result(optimizer=None, optimum=(0.35263862747936875, 0.35596858259736497))
Case: Pruned, Problem: (8, 'L', 6, 'R', 1, 'L'):000011100000001011101111000000110111100000110001100111001100000010101111001110110010101110000111100010001111100100011100101
Worker 0 Idle Result(optimizer=None, optimum=

Case: Pruned, Problem: (8, 'L', 5, 'R', 6, 'R', 2, 'L', 11, 'R'):000000000000000000100000000000010000100000000000100001000000000010000001000000000010000010000000000000000110000000000100000
Worker 0 Idle Result(optimizer=None, optimum=(0.35263862747936875, 0.35596858259736497))
Case: Pruned, Problem: (8, 'L', 5, 'R', 6, 'R', 2, 'L', 9, 'L'):000000000000000000100001000000010001100000010000100011000100000010001001000000010010001010000000100000000110100000000100000
Worker 0 Idle Result(optimizer=None, optimum=(0.35263862747936875, 0.35596858259736497))
Case: Pruned, Problem: (8, 'L', 5, 'R', 6, 'R', 2, 'L', 10, 'L'):000000000000000000100010000000010000100000100000100101001000000010100001000000100010100010000001000000000111000000000100000
Worker 0 Idle Result(optimizer=None, optimum=(0.35263862747936875, 0.35596858259736497))
Case: Pruned, Problem: (8, 'L', 5, 'R', 6, 'R', 2, 'L', 0, 'R'):0000000000000000001000110000000100010000000000001001100000000000101010000000000000000000100000011000000

Case: Pruned, Problem: (5, 'R', 10, 'R', 4, 'R', 6, 'R'):001000101000010000000101001000000011000000010000000010000100000000001000000000010000001000000000000000000000000000000000000
Worker 0 Idle Result(optimizer=None, optimum=(0.35263862747936875, 0.35596858259736497))
Case: Pruned, Problem: (4, 'R', 10, 'R'):001000101000010000000101001000000011001000010000001010000100101000001000010100010001001000100010010000100000000001001010000
Worker 0 Idle Result(optimizer=None, optimum=(0.35263862747936875, 0.35596858259736497))
Case: Pruned, Problem: (6, 'R', 11, 'L'):011101111101110110011111011111000111000111110010000110011100011100101000001111110100101000001111100100001001101000011000000
Worker 0 Idle Result(optimizer=None, optimum=(0.35263862747936875, 0.35596858259736497))
Case: Pruned, Problem: (5, 'R', 11, 'L'):011101111101110110011111011111000111011011111010011110011101100100101000110001110100101001101001100101100001101011000000000
Worker 0 Idle Result(optimizer=None, optimum=(0.352638627

Case: Pruned, Problem: (3, 'R', 6, 'R', 11, 'L'):011101111101110110011111011111000111000111110010000000001100011100000000001110000000000000000110000000000000000000000000000
Worker 0 Idle Result(optimizer=None, optimum=(0.35263862747936875, 0.35596858259736497))
Case: Pruned, Problem: (3, 'R', 6, 'R', 10, 'L'):110111010111101111111010110111110100100111100011000001001010010100000110001010000000000100000100000000000000000000000000000
Worker 0 Idle Result(optimizer=None, optimum=(0.35263862747936875, 0.35596858259736497))
Case: Pruned, Problem: (3, 'R', 6, 'R', 5, 'R'):111111111111111111111111111111110111100011110010000001001100000100000000000000000000000000000000000000000000000000000000000
Worker 0 Idle Result(optimizer=None, optimum=(0.35263862747936875, 0.35596858259736497))
Case: Pruned, Problem: (3, 'R', 6, 'R'):111111111111111111111111111111110111100111110011000001001110011100000110001110000000000100000110000000000000000000000000000
Worker 0 Idle Result(optimizer=None, optimum=(0.352

# Running Sequential OSDT

In [ ]:
# All dependencies of this notebook

# third-party imports
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier

# local imports
from lib.osdt_classifier import OSDTClassifier
from lib.model_selection import train_cross_validate
from lib.data_processing import read_dataset

# Using COMPAS as an example
dataset = read_dataset('data/preprocessed/compas-binary.csv') 
(n, m) = dataset.shape
X = dataset.values[:,:-1]
y = dataset.values[:,-1]

hyperparameters = {
    'regularization': 0.005, # Regularization coefficient which effects the penalty on model complexity
    'max_depth': float('Inf'), # User-specified limit on the model
    'max_time': float('Inf'), # User-specified limit on the runtime 
    
    'configuration': { # More configurations around toggling optimizations and prioritization options
        'priority_metric': 'curiosity',
        'look_ahead': True,
        'support_lowerbound': True,
        'incremental_accuracy_lowerbound': True,
        'accuracy_lowerbound': True,
        'equivalent_point_lowerbound': True,
    }
}

model = OSDTClassifier(**hyperparameters)
prediction = model.predict(X)
prediction = prediction.reshape(1, n)
print('Runtime: {} Seconds'.format(time() - start))
print('Prediction: \n{}'.format(prediction))
print('Training Accuracy: {}'.format(model.score(X, y)))

# Sample Experiment